In [ ]:
import transformers
import torch
torch.cuda.get_device_name(0)

'Tesla T4'

## CHECK URL, MAIL, PHONE

In [ ]:
import sys
import os
import requests
import json

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
sys.path.append(project_root)

from CrewAI.tools.check import (
    check_url_virustotal, parse_vt_result_for_display,
    check_email_validity, parse_email_result,
    check_phone_validity, parse_phone_result
)

In [ ]:
def build_checks_summary(url=None, email=None, phone=None):
    parts = []

    if url:
        url_result = check_url_virustotal(url)
        check_url = parse_vt_result_for_display(url_result)
        parts.append(f"Kết quả kiểm tra URL: {check_url}")

    if email:
        mail_result = check_email_validity(email)
        check_mail = parse_email_result(mail_result)
        parts.append(f"Kết quả kiểm tra Mail: {check_mail}")

    if phone:
        phone_result = check_phone_validity(phone)
        check_phone = parse_phone_result(phone_result)
        parts.append(f"Kết quả kiểm tra Phone: {check_phone}")

    return parts

## Load Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "unsloth/mistral-7b-instruct-v0.2"
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    use_auth_token=True,
    torch_dtype="auto",
    device_map="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/722 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
prompt = "Bạn đã trúng thưởng 1 chiếc Iphone 16, hãy nhấn vào link để nhận thưởng"
messages = [
    {"role": "system", "content": "Bạn là 1 AI thông minh hỗ trợ phân loại tin tức real và fake. Hãy phân loại tin tức người dùng thuộc loại real hoặc fake. Nhớ chỉ cần trả lời đúng là real hoặc fake không cần giải thích thêm"},
    {"role": "user", "content": prompt}
]

In [ ]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

Fake. Apple has not released an iPhone 16 model yet.


## Testing

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import re

def extract_contact_info(text: str) -> str:
    email_pattern = r"[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+"
    url_pattern = r"https?://[^\s]+|www\.[^\s]+"
    phone_pattern = r"(\+?84|0)?\s?(\d{9,10})"

    email = ''
    phone = ''
    url = ''

    email_match = re.search(email_pattern, text)
    if email_match:
        email = email_match.group(0)

    # Tìm URL đầu tiên
    url_match = re.search(url_pattern, text)
    if url_match:
        url = url_match.group(0)

    # Tìm số điện thoại đầu tiên
    phone_match = re.search(phone_pattern, text)
    if phone_match:
        phone = "".join([g if g is not None else "" for g in phone_match.groups()]) if phone_match else ""

    return email, phone, url

def classify_news(input_text: str, check_summary: list) -> str:
    joined_check = "\n".join(check_summary)
    full_prompt = f"""Bạn là trợ lý AI có nhiệm vụ xác thực tin tức là real hay fake.

    Thông tin cần xác thực: {input_text}

    Kết quả kiểm tra bổ sung (nếu có):
    {joined_check}

    Yêu cầu:
    Chỉ trả lời duy nhất 1 trong 2 từ sau: real hoặc fake.
    Không thêm giải thích, không ghi chú, không dòng thừa.


    Kết luận:"""

    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(**inputs, max_new_tokens=20)
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    return response.strip()

In [ ]:
# ==== TEST ====
input_text = "TẬP ĐOÀN YONGYI CẦN TUYỂN GẤP CÁC VỊ TRÍ : SALE - HR - HẬU ĐÀI - TELESALE - PHIÊN DỊCH  HỘ TRỢ VÉ MÁY BAY, HỘ CHIẾU, ĐƯA ĐÓN ỨNG VIÊN CHI TIẾT IB TELE : https://thuedientu.gdt.gov.vn."
email, phone, url = extract_contact_info(input_text)

check_summary = build_checks_summary(url, email, phone)
print("Check Summary:", check_summary)

final_label = classify_news(input_text, check_summary)
print("\n Kết luận cuối cùng:", final_label)

Check Summary: ["Kết quả kiểm tra URL: {'url': 'https://thuedientu.gdt.gov.vn/', 'harmless': 70, 'malicious': 0, 'suspicious': 0, 'undetected': 27, 'overall': 'An toàn'}"]

 Kết luận cuối cùng: real.


## Run with test_dataset

In [ ]:
import pandas as pd
import json

df = pd.read_csv('/content/test.csv')
llm_outputs = []

for i, row in df.iterrows():
    input_text = row['text']

    try:
        email, phone, url = extract_contact_info(input_text)
        check_summary = build_checks_summary(url, email, phone)

        # Phân loại
        final_label = classify_news(input_text, check_summary)
        llm_outputs.append(final_label)
        print("Kết luận:", final_label)

    except Exception as e:
        error_msg = f"Error: {str(e)}"
        llm_outputs.append(error_msg)
        print("❌ Lỗi xử lý:", error_msg)

# Ghi kết quả vào cột mới và lưu file
df['Mistral-7B'] = llm_outputs
df.to_csv('test_Mistral-7B.csv', index=False)

print("\nĐã xử lý xong toàn bộ test.csv và lưu kết quả.")